In [1]:
cd ../

/wk171/ytkuo/TCRI_reanalysis


In [2]:
%env CUDA_VISIBLE_DEVICES=-1

env: CUDA_VISIBLE_DEVICES=-1


In [3]:
import tensorflow as tf
tf.config.optimizer.set_jit(True)
from modules.data_handler import ascii_array_to_string
import pandas as pd
import numpy as np

2023-06-25 17:10:03.347220: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [4]:
# fet experiment settings from yaml file
from modules.experiment_helper import parse_experiment_settings

experiment_path = 'experiments/cnn-two-stage.yml'
experiment_settings = parse_experiment_settings(experiment_path)

In [5]:
# load dataset
from modules.data_handler import get_tensorflow_datasets

datasets = get_tensorflow_datasets(**experiment_settings['data'])

2023-06-25 17:10:04.716441: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-06-25 17:10:04.770588: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-06-25 17:10:04.770616: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: ubuntu
2023-06-25 17:10:04.770623: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: ubuntu
2023-06-25 17:10:04.770721: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 460.39.0
2023-06-25 17:10:04.770744: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 460.39.0
2023-06-25 17:10:04.770750: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 460.39.0
2023-06-25 17:10:04.771310: I tensorflow/core/platform/cpu_feature_gua

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


In [6]:
# load trained model
from modules.model_constructor import create_model_by_experiment_path

model = create_model_by_experiment_path(experiment_path, version='best-MSE')

saved_models/cnn-two-stage/best-MSE


In [7]:
df = pd.DataFrame(columns=['now', '24h_Vmax', 'pred_24h_Vmax', 'starting_frame'])

data_list = [(list(map(ascii_array_to_string, frame_ID_ascii.numpy())), labels.numpy(), 
              model(image_sequences, feature, dv3h).numpy()[:,0], labels.numpy() - dV.numpy()) 
             for image_sequences, labels,_,_, feature,_,_, frame_ID_ascii, dv3h, dV in datasets['test']]

data_dict = {'now': [], '24h_Vmax': [], 'pred_24h_Vmax': [], 'starting_frame': []}

for starting_frame, V24, pred, now in data_list:
    data_dict['now'].extend(now)
    data_dict['24h_Vmax'].extend(V24)
    data_dict['pred_24h_Vmax'].extend(pred)
    data_dict['starting_frame'].extend(starting_frame)

df = pd.DataFrame(data_dict)

2023-06-25 17:10:05.622235: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2023-06-25 17:10:05.623058: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3199960000 Hz
2023-06-25 17:10:05.634892: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1653] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA_FLAGS) or set the envvar XLA_FLAGS=--xla_hlo_profile.


In [8]:
df['now'] = df['now']*28.126 + 39.669
df['24h_Vmax'] = df['24h_Vmax']*28.126 + 39.669
df['pred_24h_Vmax'] = df['pred_24h_Vmax']*28.126 + 39.669
df = df.sort_values(['starting_frame']).reset_index(drop=True).round(2)

In [9]:
df.head(5)

,now,24h_Vmax,pred_24h_Vmax,starting_frame
0,16.12,33.52,33.57,201901E_2019062521
1,18.12,35.25,31.90,201901E_2019062600
2,18.30,37.86,31.12,201901E_2019062603
3,18.68,39.13,29.58,201901E_2019062606
4,22.16,38.69,32.91,201901E_2019062609


In [10]:
df.to_csv('./saved_csv/cnn_only_two_stage_3_encoded_label_max.csv', index = False)